# Using Fine-Tuned Stable Diffusion Model (amr3303)

This notebook demonstrates how to use the fine-tuned Stable Diffusion model from amr3303 to generate high-quality images. We'll explore different parameters and techniques to achieve optimal results similar to checkpoint generations.

## Setup and Installation

First, let's install the necessary dependencies if they're not already installed.

In [ ]:
# Uncomment and run if you need to install dependencies
# !pip install diffusers transformers accelerate safetensors

## Loading the Model

Now, let's load the fine-tuned Stable Diffusion model from Hugging Face Hub. You'll need to authenticate with your Hugging Face token.

In [ ]:
from diffusers import DiffusionPipeline
import torch
from huggingface_hub import login

# Login to Hugging Face Hub
# Replace with your own token if needed
login()

# Load the model and move it to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the fine-tuned model
pipe = DiffusionPipeline.from_pretrained("amr3303/fine-tuned-sd-amr", torch_dtype=torch.float16)
pipe = pipe.to(device)

# Enable memory efficient attention if using PyTorch 2.0
if hasattr(pipe, "enable_xformers_memory_efficient_attention"):
    pipe.enable_xformers_memory_efficient_attention()
elif hasattr(pipe, "enable_attention_slicing"):
    pipe.enable_attention_slicing()


## Basic Image Generation

Let's start with a basic image generation using default parameters.

In [ ]:
import matplotlib.pyplot as plt

def display_images(images, titles=None, cols=3, figsize=(15, 10)):
    """Display a list of images in a grid."""
    rows = (len(images) + cols - 1) // cols
    fig = plt.figure(figsize=figsize)
    for i, image in enumerate(images):
        ax = fig.add_subplot(rows, cols, i + 1)
        if titles is not None:
            ax.set_title(titles[i])
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Basic prompt
prompt = "A beautiful portrait of a woman with long hair, detailed features, high quality, photorealistic"

# Generate image with default parameters
image = pipe(prompt).images[0]

# Display the image
display_images([image], ["Basic Generation"])


## Optimizing Generation Parameters

Now, let's experiment with different parameters to achieve better results:

1. **Guidance Scale**: Controls how closely the image follows the prompt (higher values = more adherence)
2. **Number of Steps**: More steps generally produce better quality but take longer
3. **Scheduler**: Different schedulers can produce different results
4. **Seed**: Setting a seed allows for reproducible results

In [ ]:
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler

# Test different guidance scales
guidance_scales = [7.0, 8.5, 10.0]
images_guidance = []
titles_guidance = []

# Set a seed for reproducibility
generator = torch.Generator(device=device).manual_seed(42)

for guidance_scale in guidance_scales:
    image = pipe(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=50,  # Increased steps for better quality
        generator=generator,
    ).images[0]
    
    images_guidance.append(image)
    titles_guidance.append(f"Guidance Scale: {guidance_scale}")

# Display the results
display_images(images_guidance, titles_guidance)


In [ ]:
# Test different schedulers
schedulers = [
    ("DPM++ 2M", DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)),
    ("Euler", EulerDiscreteScheduler.from_config(pipe.scheduler.config)),
    ("Euler a", EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config))
]

images_scheduler = []
titles_scheduler = []

# Use the best guidance scale from previous experiment
best_guidance_scale = 8.5  # Adjust based on your preference from previous results

for name, scheduler in schedulers:
    # Update the scheduler
    pipe.scheduler = scheduler
    
    # Generate image
    image = pipe(
        prompt,
        guidance_scale=best_guidance_scale,
        num_inference_steps=50,
        generator=torch.Generator(device=device).manual_seed(42),
    ).images[0]
    
    images_scheduler.append(image)
    titles_scheduler.append(f"Scheduler: {name}")

# Display the results
display_images(images_scheduler, titles_scheduler)


## Advanced Prompt Engineering

The quality of the generated images heavily depends on the prompts. Let's experiment with different prompt techniques to get the best results from this model.

In [ ]:
# Select the best scheduler based on previous results
# Adjust this based on your preference from the previous experiment
best_scheduler_name = "DPM++ 2M"  # Example, change as needed
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Test different prompt techniques
prompts = [
    "A beautiful portrait of a woman with long hair",  # Basic prompt
    "A beautiful portrait of a woman with long hair, detailed features, high quality, photorealistic, 8k, professional photography",  # Detailed prompt with quality descriptors
    "A beautiful portrait of a woman with long hair, detailed features, high quality, photorealistic, 8k, professional photography, soft lighting, shallow depth of field, bokeh, award-winning, masterpiece",  # Advanced prompt with artistic descriptors
]

images_prompt = []
titles_prompt = []

for i, prompt in enumerate(prompts):
    image = pipe(
        prompt,
        guidance_scale=best_guidance_scale,
        num_inference_steps=50,
        generator=torch.Generator(device=device).manual_seed(42),
    ).images[0]
    
    images_prompt.append(image)
    titles_prompt.append(f"Prompt Technique {i+1}")

# Display the results
display_images(images_prompt, titles_prompt)


## Negative Prompts

Negative prompts tell the model what to avoid in the generated image. This can significantly improve the quality of the output.

In [ ]:
# Best prompt from previous experiment
best_prompt = "A beautiful portrait of a woman with long hair, detailed features, high quality, photorealistic, 8k, professional photography, soft lighting, shallow depth of field, bokeh, award-winning, masterpiece"

# Test different negative prompts
negative_prompts = [
    None,  # No negative prompt
    "blurry, low quality, distorted, deformed features",  # Basic negative prompt
    "blurry, low quality, distorted, deformed features, bad anatomy, disfigured, poorly drawn face, mutation, mutated, extra limb, ugly, poorly drawn hands, missing limb, floating limbs, disconnected limbs, malformed hands, out of focus, long neck, long body, text, watermark, signature",  # Comprehensive negative prompt
]

images_negative = []
titles_negative = []

for i, negative_prompt in enumerate(negative_prompts):
    image = pipe(
        best_prompt,
        negative_prompt=negative_prompt,
        guidance_scale=best_guidance_scale,
        num_inference_steps=50,
        generator=torch.Generator(device=device).manual_seed(42),
    ).images[0]
    
    images_negative.append(image)
    titles_negative.append(f"Negative Prompt {i+1}")

# Display the results
display_images(images_negative, titles_negative)


## Optimized Generation

Now, let's combine all the best parameters and techniques to generate the highest quality images.

In [ ]:
# Optimal parameters based on previous experiments
optimal_prompt = "A beautiful portrait of a woman with long hair, detailed features, high quality, photorealistic, 8k, professional photography, soft lighting, shallow depth of field, bokeh, award-winning, masterpiece"

optimal_negative_prompt = "blurry, low quality, distorted, deformed features, bad anatomy, disfigured, poorly drawn face, mutation, mutated, extra limb, ugly, poorly drawn hands, missing limb, floating limbs, disconnected limbs, malformed hands, out of focus, long neck, long body, text, watermark, signature"

# Generate multiple images with different seeds
seeds = [42, 123, 456, 789, 1024]
images_optimal = []
titles_optimal = []

for seed in seeds:
    image = pipe(
        optimal_prompt,
        negative_prompt=optimal_negative_prompt,
        guidance_scale=best_guidance_scale,
        num_inference_steps=50,
        generator=torch.Generator(device=device).manual_seed(seed),
    ).images[0]
    
    images_optimal.append(image)
    titles_optimal.append(f"Seed: {seed}")

# Display the results
display_images(images_optimal, titles_optimal, cols=2, figsize=(12, 15))


## Saving Generated Images

Let's save our best generated images to disk.

In [ ]:
import os

# Create output directory if it doesn't exist
output_dir = "generated_images"
os.makedirs(output_dir, exist_ok=True)

# Save the optimal images
for i, image in enumerate(images_optimal):
    image_path = os.path.join(output_dir, f"optimal_image_{i+1}.png")
    image.save(image_path)
    print(f"Saved image to {image_path}")


## Conclusion

In this notebook, we've explored how to use the fine-tuned Stable Diffusion model from amr3303 to generate high-quality images. We've experimented with different parameters and techniques to achieve optimal results:

1. **Best Parameters**:
   - Guidance Scale: 8.5 (adjust based on your preference)
   - Number of Steps: 50
   - Scheduler: DPM++ 2M (adjust based on your preference)

2. **Prompt Engineering**:
   - Use detailed prompts with quality descriptors
   - Include artistic elements like lighting and depth of field
   - Use comprehensive negative prompts to avoid common issues

3. **Tips for Best Results**:
   - Experiment with different seeds to find the best results
   - Adjust guidance scale based on your specific prompt
   - Use negative prompts to avoid common artifacts

This model produces excellent results when properly prompted and configured. Feel free to experiment with different prompts and parameters to achieve your desired output.